In [5]:
import csv
from pathlib import Path

from ns_vfs.common.utility import save_frames
from ns_vfs.config.loader import load_config
from ns_vfs.data.frame import BenchmarkLTLFrame, FramesofInterest
from ns_vfs.frame_searcher import FrameSearcher
from ns_vfs.model.vision.grounding_dino import GroundingDino
from ns_vfs.processor.benchmark_video_processor import BenchmarkVideoFrameProcessor
from ns_vfs.video_to_automaton import VideotoAutomaton
from common import get_available_benchmark_video
from ns_vfs.model.vision.yolo import Yolo

**Global Variable**

In [3]:
config = load_config()
benchmark_frame_video_root_dir = Path(
    "/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/benchmark_frame_video/"
)
benchmark_image_set_dir = [x for x in benchmark_frame_video_root_dir.iterdir() if x.is_dir()]
cv_model_list = ["grounding_dino", "yolo"]

**Local Variable for the experiment**

In [ ]:
"""
It will go over all available benchmark video and search for frame of interest for each cv detection model.
"""
for benchmark_name_dir in benchmark_image_set_dir:
    ltl_video_dir_set = [x for x in benchmark_name_dir.iterdir() if x.is_dir()]
    if len(ltl_video_dir_set) > 0:
        print(f"--processing {benchmark_name_dir.name}--")
        print(f"number of ltl rule: {len(ltl_video_dir_set)}")
        for ltl_video_dir in ltl_video_dir_set:
            benchmark_video_file_list = get_available_benchmark_video(ltl_video_dir)
            print(f"number of examples of {ltl_video_dir.name}: {len(benchmark_video_file_list)}")

            for benchmark_video_file in benchmark_video_file_list:
                for cv_model in cv_model_list:
                    if cv_model == "yolo":
                        cv_detection_model = Yolo(config=config.YOLO,
                                                  weight_path=config.YOLO.YOLO_CHECKPOINT_PATH)
                    elif cv_model == "grounding_dino":
                        cv_detection_model = GroundingDino(
                                config=config.GROUNDING_DINO,
                                weight_path=config.GROUNDING_DINO.GROUNDING_DINO_CHECKPOINT_PATH,
                                config_path=config.GROUNDING_DINO.GROUNDING_DINO_CONFIG_PATH,
                            )
                    benchmark_video_processor = BenchmarkVideoFrameProcessor(
                        video_path=benchmark_video_file,
                        artifact_dir=config.VERSION_AND_PATH.ARTIFACTS_PATH,
                        manual_confidence_probability=1.0,
                    )

                    benchmark_img_frame: BenchmarkLTLFrame = benchmark_video_processor.benchmark_image_frames

                    video_automata_builder = VideotoAutomaton(
                        detector=cv_detection_model,
                        video_processor=benchmark_video_processor,
                        artifact_dir=config.VERSION_AND_PATH.ARTIFACTS_PATH,
                        proposition_set=benchmark_img_frame.proposition,
                        save_annotation=False,  # TODO: Debug only
                        save_image=False,  # TODO: Debug only
                        ltl_formula=f"P>=0.80 [{benchmark_img_frame.ltl_formula}]",
                        verbose=False,
                    )
                    frame_sercher = FrameSearcher(
                        video_automata_builder=video_automata_builder,
                        video_processor=benchmark_video_processor,
                    )

                    frame_of_interest = frame_sercher.search()